In [9]:
import sys
sys.path.append('/Users/bence/code/liouvillian_metro/')

import numpy as np
import qutip as qt
import matplotlib.pyplot as plt
from scipy.linalg import expm
from scipy.sparse.linalg import eigsh
import pickle
from copy import deepcopy
from qiskit.quantum_info import DensityMatrix, Statevector, partial_trace, state_fidelity
import time

from oft_n_liouv_np import oft
from tools.classical import find_ideal_heisenberg, trotter_step_heisenberg_qt, ham_evol_qt, hamiltonian_matrix

#* Benchmarking says, for no parallelization, 15-16 estimating qubits max, 10-12 system qubits max
#* 12 - 13 would take 2-3 days without GPU or cluster
#* 13 - 14 would take 33 days without GPU or cluster -> with GPU or cluster maybe 50x so within reach
#* We can compute a spectral gap of 8 qubit Liouv max, 9 qubits needs half a TB of memory
np.random.seed(667)
# 3-6, 4-8, 5-9, 6-12, 7-14, 8-16, 9-18 (<- perfect spectra); 10 - 20 (6+ mins, we can't do that, maybe with parallelization)
num_qubits = 4
num_energy_bits = 14
delta = 0.01
eps = 0.1
sigma = 5
bohr_bound = 0 #2 ** (-num_energy_bits + 1)
beta = 1
eig_index = 2
mix_time = 4

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, [1, 1, 1, 1], signed=False, for_oft=True)
# eigenstate_with_scipy = eigsh(hamiltonian.qt.full(), k=1, sigma=hamiltonian.spectrum[eig_index])[1].flatten()
# initial_state = eigenstate_with_scipy / np.linalg.norm(eigenstate_with_scipy)
initial_state = hamiltonian.eigvecs[:, eig_index]

initial_dm = DensityMatrix(initial_state).data

N = 2**num_energy_bits
N_labels = np.arange(N / 2, dtype=int)
N_labels_neg = np.arange(- N / 2, 0, dtype=int)
N_labels = np.concatenate((N_labels, N_labels_neg))
time_labels = N_labels
phase = 17

site_list = [qt.qeye(2) for _ in range(num_qubits)]
x_jump_ops = []
for q in range(num_qubits):
    site_list[q] = qt.sigmax()
    x_jump_ops.append(qt.tensor(site_list).full())

rand_jump_index = np.random.randint(0, len(x_jump_ops))
jump_op = x_jump_ops[rand_jump_index]

gauss = lambda t: np.exp(-t**2 / (4 * sigma**2))
gauss_values = gauss(time_labels)
normalized_gauss_values_bin_order = (gauss_values / np.sqrt(np.sum(gauss_values**2)))

# if hamiltonian is None and trotter is not None:  #* Trotterized
#     time_evolution = lambda n: np.linalg.matrix_power(trotter, n)
# elif hamiltonian is not None and trotter is None:  #* Exact
# time_evolution_fn = lambda t: expm(1j * t * hamiltonian.qt.full())

Original spectrum:  [-8.0863 -5.2361 -4.     -3.2361 -1.4863 -1.2361 -0.7639 -0.      0.7639
  1.2248  1.2361  2.      3.2361  4.3478  5.2361  6.    ]
Ideal spectrum:  [0.     0.0911 0.1305 0.1549 0.2108 0.2188 0.2339 0.2583 0.2827 0.2975
 0.2978 0.3222 0.3617 0.3972 0.4256 0.45  ]
Nonrescaled coefficients:  [1, 1, 1, 1]
Rescaled coefficients:  [0.03194602 0.03194602 0.03194602 0.03194602]


In [10]:

boltzmann = lambda beta, energy: np.min([1, np.exp(-beta * energy)]) #! change it to new paper's way
boltzmann_vectorized = np.vectorize(boltzmann)
#* Exact
evolved_dm = deepcopy(initial_dm)

energy_factors = 2 * np.pi * N_labels / N
boltzmann_factors = boltzmann_vectorized(beta, energy_factors)
boltzmann_factors

array([1.        , 0.9984672 , 0.99693674, ..., 1.        , 1.        ,
       1.        ])

In [3]:
# decompose hamiltonian to diag form
# diag_decomposed_ham = hamiltonian.eigvecs @ np.diag(hamiltonian.spectrum) @ hamiltonian.eigvecs.T.conj()
# print(np.allclose(diag_decomposed_ham, hamiltonian.qt.full()))

diag_ham = np.diag(hamiltonian.spectrum)

# time_evolution_fn = lambda t: expm(1j * t * hamiltonian.qt.full())
time_evolution_fn = lambda t: hamiltonian.eigvecs @ np.diag(np.exp(1j * t * hamiltonian.spectrum)) @ hamiltonian.eigvecs.T.conj()


In [4]:
diag_elements_for_all_times = np.exp(1j * time_labels[:, np.newaxis] * hamiltonian.spectrum)
print(diag_elements_for_all_times.shape)

(128, 64)


In [5]:
phase_factors = np.exp(- 1j * 2 * np.pi * phase * time_labels / N)
t1 = time.time()
# print(jump_op.shape)

oft_op_np = np.einsum('i, i, ja, ia, ak, km, mb, ib, bn  -> jn', 
                      normalized_gauss_values_bin_order, phase_factors, hamiltonian.eigvecs, diag_elements_for_all_times,
                      hamiltonian. eigvecs.conj().T, jump_op, hamiltonian.eigvecs, diag_elements_for_all_times.conj(),
                      hamiltonian.eigvecs.conj().T, 
                      optimize=True) / np.sqrt(N)

print(f'Time to build the OFT operator with einsum: {time.time() - t1:.2f} s')

Time to build the OFT operator with einsum: 0.00 s


In [6]:
# time_evolution_fn = lambda t: expm(1j * t * hamiltonian.qt.full())
# t0 = time.time()
# oft_op = np.zeros_like(jump_op, dtype=np.complex128)

# for i, t in enumerate(time_labels):
#     oft_op += (np.exp(-1j * 2 * np.pi * phase * t / N)  #!
#                 * normalized_gauss_values_bin_order[i] 
#                 * time_evolution_fn(t) @ jump_op @ time_evolution_fn(-t))
    
# oft_op /= np.sqrt(N)
# print(f'Time to build the OFT operator: {time.time() - t0:.2f} s')

Time to build the OFT operator: 102.37 s


In [7]:

# print(oft_op_np.shape)
# np.allclose(oft_op, oft_op_np, atol=1e-7)

True